1. Visualize explanations for all test trials for all tested method
2. Run AI-only

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

import torch.nn.functional as F
import torchvision
from torchvision import models
from torchvision import transforms

import torch
import torchvision

torch.set_num_threads(1)
torch.manual_seed(0)
np.random.seed(0)

from torchvision.models import *
from visualisation.core.utils import device, image_net_postprocessing
from torch import nn
from operator import itemgetter
from visualisation.core.utils import imshow
from IPython.core.debugger import Tracer

NN_flag = False

In [3]:
layer = 4
if NN_flag:
    feature_extractor = nn.Sequential(*list(resnet34(pretrained=True).children())[:layer-6]).to(device)
    
model = resnet34(pretrained=True).to(device)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [4]:
# print(feature_extractor)

In [5]:
# %matplotlib notebook 
import glob
import matplotlib.pyplot as plt
import numpy as np
import torch 
from utils import *
from PIL import Image

plt.rcParams["figure.figsize"]= 16,8


def make_dir(path):
    if os.path.isdir(path) == False:
        os.mkdir(path)

In [6]:
import glob
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.pyplot import imshow

from visualisation.core.utils import device 
from PIL import Image
from torchvision.transforms import ToTensor, Resize, Compose, ToPILImage
from visualisation.core import *
from visualisation.core.utils import image_net_preprocessing

size = 224

# Pre-process the image and convert into a tensor
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(256),
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
])

img_num = 50

# methods = ['Conf', 'GradCAM', 'EP', 'SHAP', 'NNs', 'PoolNet', 'AIonly']
methods = ['Conf', 'GradCAM', 'EP', 'NNs', 'PoolNet', 'AIonly']

task = 'Natural'
# Adversarial_Nat

output_path = '/home/dexter/Downloads/Human_experiments/Visualization'
dataset_path = '/home/dexter/Downloads/Human_experiments/Dataset'
make_dir('{}/{}'.format(output_path,task))
make_dir('{}/{}/Conf/'.format(output_path,task))
make_dir('{}/{}/GradCAM/'.format(output_path,task))
make_dir('{}/{}/EP/'.format(output_path,task))
# make_dir('Pilot_study2/{}/SHAP/'.format(task))
make_dir('{}/{}/PoolNet/'.format(output_path,task))
make_dir('{}/{}/NNs/'.format(output_path,task))
# test_image_paths = glob.glob('/home/dexter/Downloads/Pilot_study2/{}/mixed_images/*.*'.format(task))
corrects_bin1 = glob.glob('{}/{}/correct_bin1_images/*.*'.format(dataset_path,task))
wrongs_bin1 = glob.glob('{}/{}/wrong_bin1_images/*.*'.format(dataset_path,task))
corrects_bin2 = glob.glob('{}/{}/correct_bin2_images/*.*'.format(dataset_path,task))
wrongs_bin2 = glob.glob('{}/{}/wrong_bin2_images/*.*'.format(dataset_path,task))
corrects_bin3 = glob.glob('{}/{}/correct_bin3_images/*.*'.format(dataset_path,task))
wrongs_bin3 = glob.glob('{}/{}/wrong_bin3_images/*.*'.format(dataset_path,task))
# print(test_image_paths)
# test_inputs = list()
test_images = list()

modified_img = list()


In [7]:
# Added for loading ImageNet classes
def load_imagenet_label_map():
    """
    Load ImageNet label dictionary.
    return:
    """

    input_f = open("input_txt_files/imagenet_classes.txt")
    label_map = {}
    for line in input_f:
        parts = line.strip().split(": ")
        (num, label) = (int(parts[0]), parts[1].replace('"', ""))
        label_map[num] = label

    input_f.close()
    return label_map


# Added for loading ImageNet classes
def load_imagenet_id_map():
    """
    Load ImageNet ID dictionary.
    return;
    """

    input_f = open("input_txt_files/synset_words.txt")
    label_map = {}
    for line in input_f:
        parts = line.strip().split(" ")
        (num, label) = (parts[0], ' '.join(parts[1:]))
        label_map[num] = label

    input_f.close()
    return label_map

def convert_imagenet_label_to_id(label_map, key_list, val_list, prediction_class):
    """
    Convert imagenet label to ID: for example - 245 -> "French bulldog" -> n02108915
    :param label_map:
    :param key_list:
    :param val_list:
    :param prediction_class:
    :return:
    """
    class_to_label = label_map[prediction_class]
    prediction_id = key_list[val_list.index(class_to_label)]
    return prediction_id


# gt_dict = load_imagenet_validation_gt()
id_map = load_imagenet_id_map()
label_map = load_imagenet_label_map()

key_list = list(id_map.keys())
val_list = list(id_map.values())

def convert_imagenet_id_to_label(label_map, key_list, val_list, class_id):
    """
    Convert imagenet label to ID: for example - n02108915 -> "French bulldog" -> 245
    :param label_map:
    :param key_list:
    :param val_list:
    :param prediction_class:
    :return:
    """
    return key_list.index(str(class_id))

from torchray.attribution.extremal_perturbation import extremal_perturbation, contrastive_reward
from torchray.attribution.grad_cam import grad_cam
import PIL.Image

def get_EP_saliency_maps(model, path):
        img_index = (path.split('.jpeg')[0]).split('images/')[1]
        img = PIL.Image.open(path)
        x = transform(img).unsqueeze(0).to(device)
        out = model(x)
        p = torch.nn.functional.softmax(out, dim=1)
        score, index = torch.topk(p, 1)
        category_id_1 = index[0][0].item()
        
        gt_label_id = img_index.split('val_')[1][9:18]
        input_prediction_id = convert_imagenet_label_to_id(label_map, key_list, val_list, category_id_1)

        masks, energy = extremal_perturbation(
            model, x, category_id_1,
            areas=[0.025, 0.05, 0.1, 0.2],
            num_levels=8,
            step=7,
            sigma=7 * 3,
            max_iter=800,
            debug=False,
            jitter=True,
            smooth=0.09,
            perturbation='blur'
        )
        saliency = masks.sum(dim=0, keepdim=True)
        saliency = saliency.detach()
        saliency_t = saliency.cpu().detach().numpy()[0, 0, :]
        
        saliency_path = 'saliency_maps/EP_resnet34_{}/'.format(task)
        if not (os.path.exists(saliency_path)):
            os.mkdir(saliency_path)
        
#         # Save saliency maps only when the image is correctly classified
#         if gt_label_id == input_prediction_id:
#             np.save(os.path.join(saliency_path, "{}.npy".format(img_index)), saliency_t)

        return (saliency[0].to('cpu')), (masks[0].to('cpu')), (masks[1].to('cpu')), (masks[2].to('cpu')) , (masks[3].to('cpu'))

In [8]:
# import os
import os.path
from visualisation.core.utils import tensor2cam
postprocessing_t = image_net_postprocessing
import cv2 as cv
import sys

imagenet_train_path = '/home/dexter/Downloads/train'

## Creating colormap
cMap = 'Reds'

id_list= list()
conf_dict = {}
eps=1e-5
cnt = 0
K = 3 # Change to your expected number of nearest neighbors

In [9]:
method = 'Conf'
test_image_paths = corrects_bin1[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
                    wrongs_bin1[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
                    corrects_bin2[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
                    wrongs_bin2[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
                    corrects_bin3[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
                    wrongs_bin3[methods.index(method)*img_num:methods.index(method)*img_num + img_num]

# test_image_paths = corrects_bin1[methods.index(method)*img_num + img_num:] + \
#                     wrongs_bin1[methods.index(method)*img_num + img_num:] + \
#                     corrects_bin2[methods.index(method)*img_num + img_num:] + \
#                     wrongs_bin2[methods.index(method)*img_num + img_num:] + \
#                     corrects_bin3[methods.index(method)*img_num + img_num:] + \
#                     wrongs_bin3[methods.index(method)*img_num + img_num:]

In [ ]:
# for idx, image_path in enumerate(test_image_paths[methods.index(method)*img_num:methods.index(method)*img_num + img_num]):
for idx, image_path in enumerate(test_image_paths):
#     print(image_path)
#     if 'n02106166_ILSVRC2012_val_00043367' not in image_path:
#         continue
    print(idx + 1)
    distance_dict = dict()
    neighbors = list()
    categories_confidences = list()
    confidences= list ()

    img = Image.open(image_path)
    
    if NN_flag:
        embedding = feature_extractor(transform(img).unsqueeze(0).to(device)).flatten(start_dim=1) 

    input_image = img.resize((size,size), Image.ANTIALIAS)
    image_name = (image_path.split('.jpeg')[0]).split('images/')[1]
    print(image_name)

    # Get the ground truth of the input image
    gt_label_id = image_path.split('val_')[1][9:18]

    
    gt_label = id_map.get(gt_label_id)
    id = key_list.index(gt_label_id)
    gt_label = gt_label.split(',')[0]



    # Get the prediction for the input image
    img = Image.open(image_path)
    x = transform(img).unsqueeze(0).to(device)
    out = model(x)
    p = torch.nn.functional.softmax(out, dim=1)
    score, index = torch.topk(p, 1)
    input_category_id = index[0][0].item()
    predicted_confidence = score[0][0].item()
    predicted_confidence = ("%.2f") %(predicted_confidence)
#     print(predicted_confidence)


    input_prediction_id = convert_imagenet_label_to_id(label_map, key_list, val_list, input_category_id)
    predicted_label = id_map.get(input_prediction_id).split(',')[0]
    predicted_label = predicted_label[0].lower() + predicted_label[1:]
    
    print(predicted_label)
    print(predicted_confidence)

    conf_dict[image_name] = predicted_confidence

    # Original image
    fig = plt.figure()
    plt.figure(figsize=(6.0,4.5), dpi=300)
    plt.axis('off')
    plt.title('{}: {}'.format(predicted_label, predicted_confidence))
    plt.imshow(input_image)
#     plt.imshow(modified_img[idx])
    plt.savefig('tmp/original.jpeg', bbox_inches='tight')
    plt.close()


    img = cv.resize(cv.imread(image_path,0),((size,size)))
    edges = cv.Canny(img,100,200)
    edges = edges - 255

    fig = plt.figure()
    plt.figure(figsize=(6.0,4.5), dpi=300)
    plt.axis('off')
    plt.imshow(edges, cmap = 'gray')
    plt.savefig('tmp/Edge.jpeg', bbox_inches='tight')
    plt.close()

#     Confidence score only
    fig = plt.figure()
    plt.figure(figsize=(6.0,4.5), dpi=300)
    plt.axis('off')
    plt.title('{}% {}'.format(int(float(predicted_confidence)*100), predicted_label))
    plt.imshow(input_image)
    plt.savefig('{}/{}/Conf/{}_{}.jpeg'.format(output_path, task, image_name, int(float(predicted_confidence)*100)), bbox_inches='tight', dpi=300)
    plt.close()



In [ ]:
method = 'GradCAM'
# test_image_paths = corrects_bin1[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     wrongs_bin1[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     corrects_bin2[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     wrongs_bin2[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     corrects_bin3[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     wrongs_bin3[methods.index(method)*img_num:methods.index(method)*img_num + img_num]
# print(len(test_image_paths))
# for idx, image_path in enumerate(test_image_paths[methods.index(method)*img_num:methods.index(method)*img_num + img_num]):
for idx, image_path in enumerate(test_image_paths):
#     if 'n02106166_ILSVRC2012_val_00043367' not in image_path:
#         continue
#     print(len(test_image_paths[methods.index(method)*img_num:methods.index(method)*img_num + img_num]))
    print(idx + 1)
    distance_dict = dict()
    neighbors = list()
    categories_confidences = list()
    confidences= list ()
    img = Image.open(image_path)
    
    if NN_flag:
        embedding = feature_extractor(transform(img).unsqueeze(0).to(device)).flatten(start_dim=1) 
#     print(test_image_paths[idx])
    input_image = img.resize((size,size), Image.ANTIALIAS)

    image_name = (image_path.split('.jpeg')[0]).split('images/')[1]
    print(image_name)

    # Get the ground truth of the input image
    gt_label_id = image_path.split('val_')[1][9:18]
    
    gt_label = id_map.get(gt_label_id)
    id = key_list.index(gt_label_id)
    gt_label = gt_label.split(',')[0]


    # Get the prediction for the input image
    img = Image.open(image_path)
    x = transform(img).unsqueeze(0).to(device)
    out = model(x)
    p = torch.nn.functional.softmax(out, dim=1)
    score, index = torch.topk(p, 1)
    input_category_id = index[0][0].item()
    predicted_confidence = score[0][0].item()
    predicted_confidence = ("%.2f") %(predicted_confidence)
#     print(predicted_confidence)


    input_prediction_id = convert_imagenet_label_to_id(label_map, key_list, val_list, input_category_id)
    predicted_label = id_map.get(input_prediction_id).split(',')[0]
    predicted_label = predicted_label[0].lower() + predicted_label[1:]
    
#     print(gt_label_id, input_prediction_id)
    
    conf_dict[image_name] = predicted_confidence

    # Original image
    fig = plt.figure()
    plt.figure(figsize=(6.0,4.5), dpi=300)
    plt.axis('off')
    plt.title('{}% {}'.format(int(float(predicted_confidence)*100), predicted_label))
    plt.imshow(input_image)
#     plt.imshow(modified_img[idx])
    plt.savefig('tmp/original.jpeg', bbox_inches='tight')
    plt.close()

    img = cv.resize(cv.imread(image_path,0),((size,size)))
    edges = cv.Canny(img,100,200)
    edges = edges - 255

    fig = plt.figure()
    plt.figure(figsize=(6.0,4.5), dpi=300)
    plt.axis('off')
    plt.imshow(edges, cmap = 'gray')
    plt.savefig('tmp/Edge.jpeg', bbox_inches='tight')
    plt.close()
    

#     GRAD-CAM
    saliency = grad_cam(
        model, x, input_category_id,
        saliency_layer='layer4',
        resize=True
    )

    img_index = (image_path.split('.jpeg')[0]).split('images/')[1]
    saliency_path = 'saliency_maps/GradCAM_resnet34_{}/'.format(task)
    saliency *= 1.0/saliency.max()
    GradCAM = saliency[0][0].cpu().detach().numpy()
    
#     # Save saliency maps only when the image is correctly classified
#     if gt_label_id == input_prediction_id:
#         np.save(os.path.join(saliency_path, "{}.npy".format(img_index)), GradCAM)

    fig = plt.figure()
    plt.figure(figsize=(6.0,4.5), dpi=300)
    plt.axis('off')
#     plt.title('Explanation')
    plt.title('GradCAM')
    plt.imshow(GradCAM, cmap=cMap, vmin=0, vmax=1)
    plt.colorbar(orientation='vertical')
    plt.savefig('tmp/heatmap.jpeg', bbox_inches='tight')
    plt.close()

    # Get overlay version
    myCmd = 'composite -blend 10 Edge.jpeg -gravity SouthWest tmp/heatmap.jpeg tmp/overlay.jpeg'
    os.system(myCmd)

#     img_path = '{}/{}/GradCAM/{}_{}.jpeg'.format(output_path, task, image_name,int(float(predicted_confidence)*100))
#     myCmd = 'montage original.jpeg overlay.jpeg -tile 2x1 -geometry +0+0 ' + img_path
#     print(myCmd)
#     os.system(myCmd)

In [ ]:
method = 'EP'
#EP 
# test_image_paths = corrects_bin1[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     wrongs_bin1[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     corrects_bin2[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     wrongs_bin2[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     corrects_bin3[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     wrongs_bin3[methods.index(method)*img_num:methods.index(method)*img_num + img_num]
# for idx, image_path in enumerate(test_image_paths[methods.index(method)*img_num:methods.index(method)*img_num + img_num]):
for idx, image_path in enumerate(test_image_paths):
#     if 'n02106166_ILSVRC2012_val_00043367' not in image_path:
#         continue
    print(idx + 1)
#     if idx == 5:
#         break
    distance_dict = dict()
    neighbors = list()
    categories_confidences = list()
    confidences= list ()
    img = Image.open(image_path)
    
    if NN_flag:
        embedding = feature_extractor(transform(img).unsqueeze(0).to(device)).flatten(start_dim=1) 

    input_image = img.resize((size,size), Image.ANTIALIAS)

    image_name = (image_path.split('.jpeg')[0]).split('images/')[1]
    print(image_name)

    # Get the ground truth of the input image
    gt_label_id = image_path.split('val_')[1][9:18]

    
    gt_label = id_map.get(gt_label_id)
    id = key_list.index(gt_label_id)
    gt_label = gt_label.split(',')[0]


    # Get the prediction for the input image
    img = Image.open(image_path)
    x = transform(img).unsqueeze(0).to(device)
    out = model(x)
    p = torch.nn.functional.softmax(out, dim=1)
    score, index = torch.topk(p, 1)
    input_category_id = index[0][0].item()
    predicted_confidence = score[0][0].item()
    predicted_confidence = ("%.2f") %(predicted_confidence)
#     print(predicted_confidence)


    input_prediction_id = convert_imagenet_label_to_id(label_map, key_list, val_list, input_category_id)
    predicted_label = id_map.get(input_prediction_id).split(',')[0]
    predicted_label = predicted_label[0].lower() + predicted_label[1:]
    
    if input_prediction_id != gt_label_id:
        continue

    conf_dict[image_name] = predicted_confidence

    # Original image
    fig = plt.figure()
    plt.figure(figsize=(6.0,4.5), dpi=300)
    plt.axis('off')
    plt.title('{}% {}'.format(int(float(predicted_confidence)*100), predicted_label))
    plt.imshow(input_image)
#     plt.imshow(modified_img[idx])
    plt.savefig('tmp/original.jpeg', bbox_inches='tight')
    plt.close()

    img = cv.resize(cv.imread(image_path,0),((size,size)))
    edges = cv.Canny(img,100,200)
    edges = edges - 255

    fig = plt.figure()
    plt.figure(figsize=(6.0,4.5), dpi=300)
    plt.axis('off')
    plt.imshow(edges, cmap = 'gray')
    plt.savefig('tmp/Edge.jpeg', bbox_inches='tight')
    plt.close()
    

    # Extremal Perturbation
    saliency,m0,m1,m2,m3 = get_EP_saliency_maps(model, image_path)
    ep_saliency_map = tensor2img(saliency)
    ep_saliency_map *= 1.0/ep_saliency_map.max()

#     ep_saliency_map = np.mean(ep_saliency_map, axis=-1)

    fig = plt.figure()
    plt.figure(figsize=(6.0,4.5), dpi=300)
    plt.axis('off')
#     plt.title('Explanation')
    plt.title('EP')
    plt.imshow(ep_saliency_map, cmap=cMap, vmin=0, vmax=1)
    plt.colorbar(orientation='vertical')
    plt.savefig('tmp/heatmap.jpeg', bbox_inches='tight')
    plt.close()

    # Get overlay version
    myCmd = 'composite -blend 10 tmp/Edge.jpeg -gravity SouthWest tmp/heatmap.jpeg tmp/overlay.jpeg'
    os.system(myCmd)

#     img_path = '{}/{}/EP/{}_{}.jpeg'.format(output_path, task, image_name, int(float(predicted_confidence)*100))
#     myCmd = 'montage original.jpeg overlay.jpeg -tile 2x1 -geometry +0+0 ' + img_path
#     os.system(myCmd)

print("Done!")
#----------------------------------------------------------------------------------------------------

In [ ]:
# method = 'SHAP'
# # SHAP
# test_image_paths = corrects_bin1[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     wrongs_bin1[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     corrects_bin2[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     wrongs_bin2[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     corrects_bin3[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     wrongs_bin3[methods.index(method)*img_num:methods.index(method)*img_num + img_num]

# # for idx, image_path in enumerate(test_image_paths[methods.index(method)*img_num:methods.index(method)*img_num + img_num]):
# for idx, image_path in enumerate(test_image_paths):
#     print(idx + 1)
#     distance_dict = dict()
#     neighbors = list()
#     categories_confidences = list()
#     confidences= list ()
#     img = Image.open(image_path)
    
#     if NN_flag:
#         embedding = feature_extractor(transform(img).unsqueeze(0).to(device)).flatten(start_dim=1) 
# #     print(test_image_paths[idx])
#     input_image = img.resize((size,size), Image.ANTIALIAS)
#     if adv_flag:
#         image_name = (image_path.split('.jpeg')[0]).split('images/')[1]
#     else:
#         image_name = (image_path.split('.jpeg')[0]).split('images/')[1]
#     print(image_name)

#     # Get the ground truth of the input image
#     if adv_flag:
#         gt_label_id = image_path.split('val_')[1][9:18]
#     else:
#         gt_label_id = image_path[-14:-5]

    
#     gt_label = id_map.get(gt_label_id)
#     id = key_list.index(gt_label_id)
#     gt_label = gt_label.split(',')[0]


#     # Get the prediction for the input image
#     img = Image.open(image_path)
#     x = transform(img).unsqueeze(0).to(device)
#     out = model(x)
#     p = torch.nn.functional.softmax(out, dim=1)
#     score, index = torch.topk(p, 1)
#     input_category_id = index[0][0].item()
#     predicted_confidence = score[0][0].item()
#     predicted_confidence = ("%.2f") %(predicted_confidence)
# #     print(predicted_confidence)


#     input_prediction_id = convert_imagenet_label_to_id(label_map, key_list, val_list, input_category_id)
#     predicted_label = id_map.get(input_prediction_id).split(',')[0]
#     predicted_label = predicted_label[0].lower() + predicted_label[1:]

#     conf_dict[image_name] = predicted_confidence

#     # Original image
#     fig = plt.figure()
#     plt.figure(figsize=(6.0,4.5), dpi=300)
#     plt.axis('off')
#     plt.title('{}: {}'.format(predicted_label, predicted_confidence))
#     plt.imshow(input_image)
# #     plt.imshow(modified_img[idx])
#     plt.savefig('original.jpeg', bbox_inches='tight')
#     plt.close()

#     img = cv.resize(cv.imread(image_path,0),((size,size)))
#     edges = cv.Canny(img,100,200)
#     edges = edges - 255

#     fig = plt.figure()
#     plt.figure(figsize=(6.0,4.5), dpi=300)
#     plt.axis('off')
#     plt.imshow(edges, cmap = 'gray')
#     plt.savefig('Edge.jpeg', bbox_inches='tight')
#     plt.close()
    

#     test_images = Image.open(image_path)
#     test_images = [transform(test_images).unsqueeze(0)]
#     test_images = [i.to(device) for i in test_images]

#     test_images_t = torch.cat(test_images)
#     shap_values = e.shap_values(test_images_t, ranked_outputs=1, output_rank_order='max') # 2xranked_outputsx1x3x224x224

#     square_img = np.sum(shap_values[0][0][0], axis=0)
#     min_val = square_img.min()
#     max_val = square_img.max()
#     if (-1*min_val) > max_val:
#         max_val = -1.0*min_val

#     square_img *= 1.0/max_val
#     fig = plt.figure()
#     plt.figure(figsize=(6.0,4.5), dpi=300)
#     plt.axis('off')
#     plt.title('Explanation')
#     plt.imshow(square_img, cmap=cMap, vmin=-1, vmax=1)

#     plt.colorbar(orientation='vertical')
#     plt.savefig('heatmap.jpeg', bbox_inches='tight')
#     plt.close()

#     # Get overlay version
#     myCmd = 'composite -blend 10 Edge.jpeg -gravity SouthWest heatmap.jpeg overlay.jpeg'
#     os.system(myCmd)

#     img_path = 'Pilot_study2/{}/SHAP/{}.jpeg'.format(task, image_name)
#     myCmd = 'montage original.jpeg overlay.jpeg -tile 2x1 -geometry +0+0 ' + img_path
#     os.system(myCmd)

#     img_index = (image_path.split('.jpeg')[0]).split('images/')[1]
#     saliency_t = (shap_values[0][0])[0, 0, :]
#     saliency_path = 'saliency_maps/SHAP/'
#     if not (os.path.exists(saliency_path)):
#         os.mkdir(saliency_path)
#     np.save(os.path.join(saliency_path, "{}_.npy".format(img_index)), saliency_t)

# #----------------------------------------------------------------------------------------------------

In [ ]:
method = 'NNs'
# NNs
# test_image_paths = corrects_bin1[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     wrongs_bin1[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     corrects_bin2[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     wrongs_bin2[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     corrects_bin3[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     wrongs_bin3[methods.index(method)*img_num:methods.index(method)*img_num + img_num]

# for idx, image_path in enumerate(test_image_paths[methods.index(method)*img_num:methods.index(method)*img_num + img_num]):
for idx, image_path in enumerate(test_image_paths):
#     if 'n02106166_ILSVRC2012_val_00043367' not in image_path:
#         continue
#     if idx == 2:
#         break
    print(idx + 1)
#     if "32048" not in image_path:
#         continue
    distance_dict = dict()
    neighbors = list()
    categories_confidences = list()
    confidences= list ()
    img = Image.open(image_path)
    
    if NN_flag:
        embedding = feature_extractor(transform(img).unsqueeze(0).to(device)).flatten(start_dim=1) 
#     print(test_image_paths[idx])
    input_image = img.resize((size,size), Image.ANTIALIAS)
    
    image_name = (image_path.split('.jpeg')[0]).split('images/')[1]
    print(image_name)

    # Get the ground truth of the input image
    gt_label_id = image_path.split('val_')[1][9:18]

    gt_label = id_map.get(gt_label_id)
    id = key_list.index(gt_label_id)
    gt_label = gt_label.split(',')[0]


    # Get the prediction for the input image
    img = Image.open(image_path)
    x = transform(img).unsqueeze(0).to(device)
    out = model(x)
    p = torch.nn.functional.softmax(out, dim=1)
    score, index = torch.topk(p, 1)
    input_category_id = index[0][0].item()
    predicted_confidence = score[0][0].item()
    predicted_confidence = ("%.2f") %(predicted_confidence)
#     print(predicted_confidence)


#     if os.path.exists('{}/{}/NNs/{}_{}.jpeg'.format(output_path, task, image_name, int(float(predicted_confidence)*100))) == True:
#         continue


    input_prediction_id = convert_imagenet_label_to_id(label_map, key_list, val_list, input_category_id)
    predicted_label = id_map.get(input_prediction_id).split(',')[0]
    predicted_label = predicted_label[0].lower() + predicted_label[1:]

    conf_dict[image_name] = predicted_confidence

    # Original image
    fig = plt.figure()
    plt.figure(figsize=(6.0,4.5), dpi=300)
    plt.axis('off')
    plt.title('{}% {}'.format(int(float(predicted_confidence)*100), predicted_label))
    plt.imshow(input_image)
#     plt.imshow(modified_img[idx])
    plt.savefig('tmp/original.jpeg', bbox_inches='tight')
    plt.close()

    img = cv.resize(cv.imread(image_path,0),((size,size)))
    edges = cv.Canny(img,100,200)
    edges = edges - 255

    fig = plt.figure()
    plt.figure(figsize=(6.0,4.5), dpi=300)
    plt.axis('off')
    plt.imshow(edges, cmap = 'gray')
    plt.savefig('tmp/Edge.jpeg', bbox_inches='tight')
    plt.close()
    
    if NN_flag:
        from utils import *
        ## Nearest Neighbors

        predicted_set_path = os.path.join(imagenet_train_path, input_prediction_id)
        predicted_set_img_paths = glob.glob(predicted_set_path + '/*.*')
        predicted_set_color_images= list()

        embedding = embedding.detach()
        embedding.to(device)
        # Build search space for nearest neighbors
        for i, path in enumerate(predicted_set_img_paths):
            img = Image.open(path)
            if img.mode != 'RGB':
                img.close()
                del img
                continue

            x = transform(img).unsqueeze(0).to(device)
            out = model(x)
            p = torch.nn.functional.softmax(out, dim=1)
            del out
            score, index = torch.topk(p, 1)
            del p
            category_id = index[0][0].item()
            del score, index
            
#             This is to avoid the confusion from crane 134 and crane 517 and to make NNs work :)
#             Because in Imagenet, annotators mislabeled 134 and 517

            if input_category_id != 134 and input_category_id != 517 and category_id != 134 and category_id != 517:
                if input_category_id != category_id:
                    continue

            f = feature_extractor(x)
            feature_vector = f.flatten(start_dim=1).to(device)
            feature_vector = feature_vector.detach()

            del f
#             del x
#             Tracer()()
            distance_dict[path] = torch.dist(embedding, feature_vector)

            del feature_vector 
            torch.cuda.empty_cache()
            img.close()
            del img
            predicted_set_color_images.append(path)

        # Get K most similar images
        res = dict(sorted(distance_dict.items(), key = itemgetter(1))[:K]) 
        print("Before...")
        print(res)
#         Tracer()()
        while distance_dict[list(res.keys())[0]] < 100:
            del distance_dict[list(res.keys())[0]]
            res = dict(sorted(distance_dict.items(), key = itemgetter(1))[:K]) 
        print("After...")
        print(res)
#         del distance_dict
        del embedding

        similar_images = list(res.keys())
#         print(similar_images)
#         print([distance_dict[x] for x in similar_images])

        for similar_image in similar_images:
            img = Image.open(similar_image)
            neighbors.append(img.resize((size,size), Image.ANTIALIAS))
            x = transform(img).unsqueeze(0).to(device)
            out = model(x)
            p = torch.nn.functional.softmax(out, dim=1)
            score, index = torch.topk(p, 1) # Get 1 most probable classes
            category_id = index[0][0].item()
            confidence = score[0][0].item()

            label = label_map.get(category_id).split(',')[0].replace("\"", "")
            label = label[0].lower() + label[1:]
            print(label + ": %.2f" %(confidence))

            categories_confidences.append((label + ": %.2f" %(confidence)))
            confidences.append(confidence)
            img.close()
            
        img_path = '{}/{}/NNs/{}_{}.jpeg'.format(output_path, task, image_name, int(float(predicted_confidence)*100))
        print(img_path)
        font = {'color': 'red',
               'weight': 'bold',
               }
        for index, neighbor in enumerate(neighbors):
            fig = plt.figure()
            plt.figure(figsize=(6.0,4.5), dpi=300)
            plt.axis('off')
#             plt.title('{}'.format(categories_confidences[index]))
            if index == 1: # Make title for the middle image (2nd image) to annotate the 3 NNs
#                 plt.title('Three examples of {}'.format(predicted_label), fontdict=font)
                plt.title('3-NNs'.format(predicted_label))
            else:
                plt.title(' ')
            plt.imshow(neighbor)
            plt.savefig('tmp/{}.jpeg'.format(index), bbox_inches='tight')
            plt.close()
            
        myCmd = 'montage tmp/[0-2].jpeg -tile 3x1 -geometry +0+0 ' + 'tmp/aggregate.jpeg'
        os.system(myCmd)

#         myCmd = 'montage original.jpeg ' + 'aggregate.jpeg' + ' -tile 2x -geometry +120+0 ' + img_path
#         os.system(myCmd)

#----------------------------------------------------------------------------------------------------

In [ ]:
method = 'PoolNet'
from shutil import copyfile, rmtree
def rm_and_mkdir(path):
    if os.path.isdir(path) == True:
        rmtree(path)
    os.mkdir(path)

# Prepare dataset
rm_and_mkdir('/home/dexter/Downloads/run-0/run-0-sal-p/')
rm_and_mkdir('/home/dexter/Downloads/PoolNet-master/data/PASCALS/Imgs/')
if os.path.isdir('/home/dexter/Downloads/PoolNet-master/data/PASCALS/test.lst'):
    os.remove('/home/dexter/Downloads/PoolNet-master/data/PASCALS/test.lst')

# test_image_paths = corrects_bin1[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     wrongs_bin1[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     corrects_bin2[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     wrongs_bin2[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     corrects_bin3[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     wrongs_bin3[methods.index(method)*img_num:methods.index(method)*img_num + img_num]

# src_paths = test_image_paths[methods.index(method)*img_num:methods.index(method)*img_num + img_num]
src_paths = test_image_paths
for src_path in src_paths:
    dst_path = '/home/dexter/Downloads/PoolNet-master/data/PASCALS/Imgs/' + src_path.split('images/')[1]
    copyfile(src_path, dst_path)

cmd = 'ls /home/dexter/Downloads/PoolNet-master/data/PASCALS/Imgs/ > /home/dexter/Downloads/PoolNet-master/data/PASCALS/test.lst'
os.system(cmd)
cmd = 'python /home/dexter/Downloads/PoolNet-master/main.py --mode=\'test\' --model=\'/home/dexter/Downloads/run-0/run-0/models/final.pth\' --test_fold=\'/home/dexter/Downloads/run-0/run-0-sal-p/\' --sal_mode=\'p\''
os.system(cmd)


npy_file_paths = glob.glob('/home/dexter/Downloads/run-0/run-0-sal-p/*.*')
print(len(npy_file_paths))
for idx, npy_file_path in enumerate(npy_file_paths): 
#     if 'n02106166_ILSVRC2012_val_00043367' not in npy_file_path:
#         continue
    print(idx)
    test_path = '/home/dexter/Downloads/PoolNet-master/data/PASCALS/Imgs' + npy_file_path.split('.npy')[0].split('sal-p')[1]
    image_name = npy_file_path.split('.npy')[0].split('sal-p')[1]
    img = Image.open(test_path)
    input_image = img.resize((size,size), Image.ANTIALIAS)
    
    # Retrieve confidence and label
    img = Image.open(test_path)
    x = transform(img).unsqueeze(0).to(device)
    out = model(x)
    p = torch.nn.functional.softmax(out, dim=1)
    score, index = torch.topk(p, 1)
    input_category_id = index[0][0].item()
    predicted_confidence = score[0][0].item()
    predicted_confidence = ("%.2f") %(predicted_confidence)
    
    if os.path.exists('{}/{}/PoolNet/{}_{}.jpeg'.format(output_path, task, image_name, int(float(predicted_confidence)*100))) == True:
        print("Continue")
        continue
    
    # For overlay heatmap
    img = cv.resize(cv.imread(test_path,0),((size,size)))
    edges = cv.Canny(img,100,200)
    edges = edges - 255

    fig = plt.figure()
    plt.figure(figsize=(6.0,4.5), dpi=300)
    plt.axis('off')
    plt.imshow(edges, cmap = 'gray')
    plt.savefig('tmp/Edge.jpeg', bbox_inches='tight')
    plt.close()
    ##############################

    
    input_prediction_id = convert_imagenet_label_to_id(label_map, key_list, val_list, input_category_id)
    predicted_label = id_map.get(input_prediction_id).split(',')[0]
    predicted_label = predicted_label[0].lower() + predicted_label[1:]
    
    # Plot input image
    fig = plt.figure()
    plt.figure(figsize=(6.0,4.5), dpi=300)
    plt.axis('off')
    plt.title('{}% {}'.format(int(float(predicted_confidence)*100), predicted_label))
    plt.imshow(input_image)
    plt.savefig('tmp/original.jpeg', bbox_inches='tight')
    plt.close()
    
    # Plot the explanation
    npy_file = np.load(npy_file_path)
    fig = plt.figure()
    plt.figure(figsize=(6.0,4.5), dpi=300)
    plt.axis('off')
#     plt.title('Explanation')
    plt.title('SOD')
    plt.imshow(npy_file, cmap=cMap, vmin=0, vmax=1)
    plt.colorbar(orientation='vertical')
    plt.savefig('tmp/heatmap.jpeg', bbox_inches='tight')
    plt.close()
    
    # Get overlay version
    myCmd = 'composite -blend 10 tmp/Edge.jpeg -gravity SouthWest tmp/heatmap.jpeg tmp/overlay.jpeg'
    os.system(myCmd)

#     image_name = image_name.split('.jpeg')[0]
#     img_path = '{}/{}/PoolNet/{}_{}.jpeg'.format(output_path, task, image_name, int(float(predicted_confidence)*100))
#     myCmd = 'montage original.jpeg overlay.jpeg -tile 2x1 -geometry +0+0 ' + img_path
#     os.system(myCmd)

In [15]:
method = 'AIonly'
# test_image_paths = test_image_paths[methods.index(method)*img_num:methods.index(method)*img_num + img_num]
# test_image_paths = corrects_bin1[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     wrongs_bin1[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     corrects_bin2[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     wrongs_bin2[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     corrects_bin3[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     wrongs_bin3[methods.index(method)*img_num:methods.index(method)*img_num + img_num]

conf_dict = dict()
print(len(test_image_paths))
for test_image_path in test_image_paths:
    img = Image.open(test_image_path)
    gt_label_id = test_image_path[-9:]
    image_name = (test_image_path.split('.JPEG')[0]).split('images/')[1]
    x = transform(img).unsqueeze(0).to(device)
    out = model(x)
    
    p = torch.nn.functional.softmax(out, dim=1)
    score, index = torch.topk(p, 1)
    input_category_id = index[0][0].item()
    input_prediction_id = convert_imagenet_label_to_id(label_map, key_list, val_list, input_category_id)

    predicted_label = id_map.get(input_prediction_id).split(',')[0]
    predicted_confidence = score[0][0].item()
    predicted_confidence = ("%.2f") %(predicted_confidence)
#     print([image_name, predicted_confidence])
    conf_dict[image_name] = predicted_confidence
print(len(conf_dict))

correct_images = glob.glob('{}/{}/correct_bin2_images/*.*'.format(dataset_path, task))
wrong_images = glob.glob('{}/{}/wrong_bin2_images/*.*'.format(dataset_path, task))

# correct_images = list()

correct_indices = list()
wrong_indices = list()

gt_dict = dict()

for correct_image in correct_images:
    image_idx = (correct_image.split('.JPEG')[0]).split('images/')[1]
    gt_dict[image_idx] = True
    correct_indices.append(image_idx)

for wrong_image in wrong_images:
    image_idx = (wrong_image.split('.JPEG')[0]).split('images/')[1]
    gt_dict[image_idx] = False
    wrong_indices.append(image_idx)
    

accuracy_T = list()
# print(gt_dict)

# T_list = list(np.arange(0.0,1.0,0.05))
T_list = [0.55]
for T in T_list:
#     print(T)
    conf_dict_T = dict()
    for key in conf_dict.keys():
        conf = float(conf_dict[key])
        if conf > T:
            conf_dict_T[key] = True
        else:
            conf_dict_T[key] = False
#     print(conf_dict_T)
    shared_items = {k: gt_dict[k] for k in gt_dict if k in conf_dict_T and gt_dict[k] == conf_dict_T[k]}
    print("Accuracy at T = {} is {}".format(T, len(shared_items)/len(test_image_paths)))
    accuracy_T.append(len(shared_items))


print("Best accuracy is {} at T = {}".format(max(accuracy_T)/len(test_image_paths), T_list[accuracy_T.index(max(accuracy_T))]))

300
300
Accuracy at T = 0.55 is 0.18666666666666668
Best accuracy is 0.18666666666666668 at T = 0.55


In [ ]:
print(task)